<h1> <b> File này dùng để điều chỉnh siêu tham số cho bài toán phân loại: </b> </h1>

- Đầu tiên sẽ cố định kích thước ảnh là (72,72), pixels_per_cell = (6,6), cells_per_block = (2,2).
- Sau đó, cố định 2 kernel là poly và linear để tìm orientations lẫn kernel cần dùng.
- Khi có được toàn bộ siêu tham số của HOG, tiến hành tìm lại siêu tham số của SVM với kernel đã tìm ở bước trước.

In [1]:
from sklearn.metrics import accuracy_score
from skimage.feature import hog
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import optuna
import cv2
import os

In [2]:
def load_images_from_folder(folder):
    """
    Load data from the specified folder.
    Input: thư mục chứa 2 lớp ảnh là Normal và HumanFace, trong mỗi lớp sẽ có nhiều ảnh thuộc lớp đó.
    Output: trả về 2 mảng numpy, 1 mảng chứa ảnh và 1 mảng chứa nhãn tương ứng với ảnh đó.
    """
    images = []
    labels = []
    label_map = {'Negative': 0, 'Positive': 1}  # Mapping labels
    for label in label_map:
        folder_path = os.path.join(folder, label)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            if not filename.endswith(('.jpg', '.jpeg', '.png')):
                continue
            img = cv2.imread(img_path)
            if img is not None:
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Chuyển đổi sang ảnh xám
                resized = cv2.resize(gray, (72, 72))  # Giả sử bạn muốn resize về kích thước 64x64
                images.append(resized)
                labels.append(label_map[label])
    return np.array(images), np.array(labels)

In [3]:
train_folder = 'Dataset/TrainDetection/TrainClassification'  
test_folder = 'Dataset/TrainDetection/TestClassification'

X_train, y_train = load_images_from_folder(train_folder)
X_test, y_test = load_images_from_folder(test_folder)

In [4]:
X_train.shape, X_test.shape

((7987, 72, 72), (1535, 72, 72))

# Tìm siêu tham số cho HOG - chỉ tìm orientations. Xác định kernel SVM tương ứng.

In [34]:
def transform(images, ori):
        hog_features = []
        for img in images:
            # Giả sử ảnh đã là grayscale và chuẩn hóa kích thước (ví dụ 64x64)
            features = hog(img,
                           orientations=ori,
                           pixels_per_cell= (6,6),
                           cells_per_block= (2,2),
                           block_norm='L2-Hys')
            hog_features.append(features)
        return np.array(hog_features)

In [35]:
def tunning_hog_by_model(_kernel):
    acc = []
    for ori in orientations:
        X_train_hog = transform(X_train, ori)
        X_test_hog = transform(X_test, ori)
        model = SVC(kernel = _kernel, class_weight = 'balanced')
        model.fit(X_train_hog, y_train)
        y_pred = model.predict(X_test_hog)
        accuracy = accuracy_score(y_pred, y_test)
        print(f"orientations = {ori} - accuracy = {accuracy}")
        acc.append(accuracy)
    return acc

In [36]:
orientations = [8, 9, 10, 11, 12]
linear_acc = tunning_hog_by_model('linear')
poly_acc = tunning_hog_by_model('poly')

linear_df = pd.DataFrame({
    'Orientations' : orientations,
    'Accuracy' : linear_acc
})

poly_df = pd.DataFrame({
    'Orientations' : orientations,
    'Accuracy' : poly_acc
})

orientations = 8 - accuracy = 0.9387622149837134
orientations = 9 - accuracy = 0.9361563517915309
orientations = 10 - accuracy = 0.9374592833876222
orientations = 11 - accuracy = 0.9374592833876222
orientations = 12 - accuracy = 0.9328990228013029
orientations = 8 - accuracy = 0.9954397394136808
orientations = 9 - accuracy = 0.9954397394136808
orientations = 10 - accuracy = 0.9973941368078176
orientations = 11 - accuracy = 0.9986970684039088
orientations = 12 - accuracy = 0.9954397394136808


In [37]:
linear_df

,Orientations,Accuracy
0,8,0.938762
1,9,0.936156
2,10,0.937459
3,11,0.937459
4,12,0.932899


In [38]:
poly_df

,Orientations,Accuracy
0,8,0.995440
1,9,0.995440
2,10,0.997394
3,11,0.998697
4,12,0.995440


# Tìm siêu tham số cho model SVM với kernel = 'poly'

In [8]:
def extract_hog_feature(image):
    features = hog(image, orientations=11, pixels_per_cell=(6, 6),
                      cells_per_block=(2, 2), block_norm='L2-Hys',
                          visualize=False)
    return features

In [6]:
def load_hog_data(input_folder):
    data = []
    label = []
    file_paths = []
    for idx, cls_name in enumerate(os.listdir(input_folder)):
        class_path = os.path.join(input_folder, cls_name)
        if os.listdir(class_path):
            for file_name in os.listdir(class_path):
                file_path = os.path.join(class_path, file_name)
                if file_path.lower().endswith(('jpg', 'png')):
                    image = cv2.imread(file_path)
                    if image is None:
                        continue
                    image = cv2.resize(image, (72,72))
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    feature = extract_hog_feature(image)
                    file_paths.append(file_path)
                    data.append(feature)
                    label.append(idx)
                    
    return np.array(data), np.array(label), file_paths

In [9]:
train_dir = 'Dataset/TrainDetection/TrainClassification'
test_dir = 'Dataset/TrainDetection/TestClassification'

X_train, y_train, file_train = load_hog_data(train_dir)
X_test, y_test, file_test = load_hog_data(test_dir)
print(f"Kich thuoc tap du lieu: Train: ({X_train.shape}, {y_train.shape}); Test: ({X_test.shape}, {y_test.shape})")

Kich thuoc tap du lieu: Train: ((7987, 5324), (7987,)); Test: ((1535, 5324), (1535,))


In [ ]:
def objective(trial):
    C = trial.suggest_categorical('C', [1, 10, 50, 100])
    gamma = trial.suggest_categorical('gamma', [0.1, 1, 10])
    degree = trial.suggest_int('degree', 3, 5)
    
    # 🔍 Huấn luyện SVM
    clf = SVC(kernel='poly', C=C, gamma=gamma, degree = degree)
    print(f"C={C}, gamma={gamma}, degree = {degree} ")
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    print(f"accuracy={accuracy}")

    return accuracy


In [27]:
class EarlyStoppingCallback(object):
    """Early stopping callback for Optuna."""

    def __init__(self, early_stopping_rounds: int, direction: str = "minimize") -> None:
        self.early_stopping_rounds = early_stopping_rounds

        self._iter = 0

        if direction == "minimize":
            self._operator = operator.lt
            self._score = np.inf
        elif direction == "maximize":
            self._operator = operator.gt
            self._score = -np.inf
        else:
            ValueError(f"invalid direction: {direction}")

    def __call__(self, study: optuna.Study, trial: optuna.Trial) -> None:
        """Do early stopping."""
        if self._operator(study.best_value, self._score):
            self._iter = 0
            self._score = study.best_value
        else:
            self._iter += 1

        if self._iter >= self.early_stopping_rounds:
            study.stop()

In [29]:
study = optuna.create_study(direction='maximize', study_name="svm_study", storage="sqlite:///svm_study.db",
load_if_exists=True)  # vì cần accuracy cao
#Cho chạy thử 50 tổ hợp, thiết lập cơ chế dừng sớm nêu có 4 lần lien tiep giong nhau
early_stopping = EarlyStoppingCallback(10, direction='maximize')
study.optimize(objective, n_trials=50, callbacks=[early_stopping])

[I 2025-05-16 22:56:40,113] A new study created in RDB with name: svm_study


C=100, gamma=1, degree = 4 


[I 2025-05-16 22:58:09,113] Trial 0 finished with value: 0.9993485342019544 and parameters: {'C': 100, 'gamma': 1, 'degree': 4}. Best is trial 0 with value: 0.9993485342019544.


accuracy=0.9993485342019544
C=10, gamma=1, degree = 4 


[I 2025-05-16 22:59:37,929] Trial 1 finished with value: 0.9993485342019544 and parameters: {'C': 10, 'gamma': 1, 'degree': 4}. Best is trial 0 with value: 0.9993485342019544.


...

accuracy=0.9986970684039088
C=1, gamma=1, degree = 5 


[I 2025-05-16 23:17:47,846] Trial 9 finished with value: 0.9980456026058632 and parameters: {'C': 1, 'gamma': 1, 'degree': 5}. Best is trial 0 with value: 0.9993485342019544.


accuracy=0.9980456026058632
C=100, gamma=0.1, degree = 4 


[I 2025-05-16 23:19:17,156] Trial 10 finished with value: 0.9993485342019544 and parameters: {'C': 100, 'gamma': 0.1, 'degree': 4}. Best is trial 0 with value: 0.9993485342019544.


accuracy=0.9993485342019544


In [39]:
# Load lại study và mô hình hóa
study = optuna.load_study(study_name="svm_study", storage="sqlite:///svm_study.db")
print("Best accuracy:", study.best_value)
print("Best parameters:", study.best_params)
optuna.visualization.plot_optimization_history(study).show()
optuna.visualization.plot_param_importances(study).show()

Best accuracy: 0.9993485342019544
Best parameters: {'C': 100, 'gamma': 1, 'degree': 4}


<h2> <b> Vậy cuối cùng, danh sách siêu tham số của ta là: </b> </h2>

- HOG: orientations=11, pixels_per_cell=(6, 6), cells_per_block=(2, 2), block_norm='L2-Hys'.
- SVM: kernel = 'poly', C = 100, gamma = 1, degree = 4